In [55]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [59]:
route_df = pd.DataFrame(columns=['outgoing_code', 'incoming_code'])
price_df = pd.DataFrame(columns=['outgoing', 'incoming', 'stops', 'duration', 'price', 'airline'])

In [4]:
chrome_options = Options()

# chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-images")

In [5]:
# Get top N airports that I wanna scrape
# Get all outgoing flight routes from: https://www.flightsfrom.com/
# Set dates for the flight
# Scrape the prices for the flights
# Save the data in a csv file
# EDA on the data

In [66]:
# Launch the browser
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(5)
wait = WebDriverWait(driver, timeout=10)

In [67]:
# Scrape top N airports and their connections
driver.get('https://www.flightsfrom.com/top-100-airports')

# Wait for table to be loaded
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'hometoplist')))

# Run through the airports
try:
    # Get the rows of the table
    rows = driver.find_elements(By.CLASS_NAME, 'hometoplist-item')

    # Since im leaving the main page after I get all the rows, the elements become stale so I save all the urls before hand
    urls = []
    for row in rows:
        urls.append(row.get_attribute('href'))

    # Go to each airport
    for i in range(5):
    # for row in rows:
        url = urls[i]
        # Set current outgoing airport
        outgoing_code = url[-3:]

        # Go to the airport page
        driver.get(url)

        # Wait for the table to be loaded
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'uk-list')))

        # Since some rows are hidden scroll and click to the show all button
        try:
            show_all_button = driver.find_element(By.CLASS_NAME, 'ff-show-all')
            banner = driver.find_element(By.ID, 'header')
            # Top banner is in the way so account for the banner height
            driver.execute_script(f'window.scroll(0, {show_all_button.location["y"] - banner.size["height"]})')
            show_all_button.click()
        except:
            pass

        # Get the outbound routes
        routes = driver.find_elements(By.CLASS_NAME, 'ff-li-list')

        # Go through each route and save it to the dataframe
        data_list = []
        for route in routes:
            incoming_code = route.find_element(By.CLASS_NAME, 'ff-row-name').text[:3]
            data_list.append({'outgoing_code': outgoing_code, 'incoming_code': incoming_code})

        data = pd.DataFrame.from_dict(data_list)
        route_df = pd.concat([route_df, data], ignore_index=True)
        # Wait for a bit for rate limits
        time.sleep(2)
except TimeoutException:
    print('TimeoutException')

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF685701502+60802]
	(No symbol) [0x00007FF68567AC02]
	(No symbol) [0x00007FF685537CE4]
	(No symbol) [0x00007FF685586D4D]
	(No symbol) [0x00007FF685586E1C]
	(No symbol) [0x00007FF6855CCE37]
	(No symbol) [0x00007FF6855AABBF]
	(No symbol) [0x00007FF6855CA224]
	(No symbol) [0x00007FF6855AA923]
	(No symbol) [0x00007FF685578FEC]
	(No symbol) [0x00007FF685579C21]
	GetHandleVerifier [0x00007FF685A0411D+3217821]
	GetHandleVerifier [0x00007FF685A460B7+3488055]
	GetHandleVerifier [0x00007FF685A3F03F+3459263]
	GetHandleVerifier [0x00007FF6857BB846+823494]
	(No symbol) [0x00007FF685685F9F]
	(No symbol) [0x00007FF685680EC4]
	(No symbol) [0x00007FF685681052]
	(No symbol) [0x00007FF6856718A4]
	BaseThreadInitThunk [0x00007FFC6BFF257D+29]
	RtlUserThreadStart [0x00007FFC6D5CAA48+40]


In [64]:
route_df

,outgoing_code,incoming_code
0,"[https://www.flightsfrom.com/SAN, https://www....",IAH
1,"[https://www.flightsfrom.com/SAN, https://www....",LGA
2,"[https://www.flightsfrom.com/SAN, https://www....",LAX
3,"[https://www.flightsfrom.com/SAN, https://www....",DEN
4,"[https://www.flightsfrom.com/SAN, https://www....",LAS
...,...,...
1145,"[https://www.flightsfrom.com/SAN, https://www....",YQB
1146,"[https://www.flightsfrom.com/SAN, https://www....",YYC
1147,"[https://www.flightsfrom.com/SAN, https://www....",BZN
1148,"[https://www.flightsfrom.com/SAN, https://www....",JAC


In [65]:
driver.quit()